In [6]:
# --- START of block copied from 01_data_prep_and_baseline.ipynb ---

# --- Setup and Imports ---
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
# Ensure all other necessary imports like tqdm, numpy are also present in the notebook

# --- Configuration ---
DATA_DIR = 'dataset'  # MUST match your actual data path
IMAGE_SIZE = 224
BATCH_SIZE = 32
VAL_SPLIT_RATIO = 0.1 
NUM_CLASSES = 1
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

# --- 1. Data Preprocessing and Loading ---

# Define Image Transforms
train_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3), 
    transforms.Resize(256),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15), 
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

# Transforms for Validation/Test Sets (No Augmentation)
test_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize(256),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

# Load Datasets
full_train_dataset = datasets.ImageFolder(
    root=f'{DATA_DIR}/train',
    transform=train_transforms 
)

# NOTE: We define test_dataset but do not use it until the final evaluation step
test_dataset = datasets.ImageFolder(
    root=f'{DATA_DIR}/test',
    transform=test_transforms
)

# Split Training Data
train_size = int((1 - VAL_SPLIT_RATIO) * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# Define the Criterion (Loss Function)
criterion = nn.BCELoss() 

print("Data loaders and criterion are now defined in this notebook's kernel.")

# --- END of copied block ---

Data loaders and criterion are now defined in this notebook's kernel.


In [7]:
## 02_model_fine_tuning.ipynb

# --- Setup and Imports ---
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm
import os
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# --- Configuration (Matching previous notebook for consistency) ---
NUM_CLASSES = 1
FINE_TUNE_LR = 1e-5 # Very low learning rate for fine-tuning
FINE_TUNE_EPOCHS = 5
UNFREEZE_LAYERS = 3 # Unfreeze the last 3 convolutional blocks/bottlenecks
MODEL_SAVE_PATH = 'final_best_model.pt'
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# --- 1. Model Definition (Same structure as previous notebook) ---

def initialize_model(num_classes):
    # Load pre-trained MobileNetV2 without weights initially
    model = models.mobilenet_v2(weights=None)

    # Replace the final fully connected layer (the "head") as done in 01_data_prep...
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Sequential(
        nn.Linear(num_ftrs, num_classes),
        nn.Sigmoid() 
    )
    return model

# --- 2. Load Baseline Model and Weights ---

# Initialize the model structure
fine_tune_model = initialize_model(NUM_CLASSES)

# Load the weights saved after training the head in Phase 1
try:
    fine_tune_model.load_state_dict(torch.load('model_artifact_phase1_head_trained.pt', map_location=DEVICE))
    print("Successfully loaded model weights from phase 1.")
except FileNotFoundError:
    print("ERROR: model_artifact_phase1_head_trained.pt not found. Ensure 01_data_prep_and_baseline.ipynb was run.")
    exit()

fine_tune_model.to(DEVICE)

# --- 3. Unfreeze Layers for Fine-Tuning ---

def setup_for_fine_tuning(model, unfreeze_count):
    # 1. Ensure ALL parameters are unfrozen first (base + head)
    for param in model.parameters():
        param.requires_grad = True

    # 2. Re-freeze the *early* layers (where generic features are learned)
    # The feature extractor is model.features (18 blocks)
    num_feature_blocks = len(model.features) 
    
    # Freeze layers from index 0 up to (and excluding) the unfreeze start point
    freeze_until_index = num_feature_blocks - unfreeze_count
    
    print(f"Total feature blocks: {num_feature_blocks}")
    print(f"Freezing blocks 0 to {freeze_until_index-1}. Unfreezing blocks {freeze_until_index} to {num_feature_blocks-1}.")

    for i in range(freeze_until_index):
        for param in model.features[i].parameters():
            param.requires_grad = False
        
    return model

# Set up the model for fine-tuning
fine_tune_ready_model = setup_for_fine_tuning(fine_tune_model, UNFREEZE_LAYERS)

# Verify trainable parameters
trainable_params = sum(p.numel() for p in fine_tune_ready_model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in fine_tune_ready_model.parameters())
print(f"Trainable parameters: {trainable_params} / Total parameters: {total_params}")


# --- 4. Fine-Tuning Training Loop ---

# Define the Loss (from previous notebook) and a NEW optimizer with low LR
criterion = nn.BCELoss() 
# Optimizing ALL trainable parameters (base + head) with a very small LR
optimizer = optim.Adam(fine_tune_ready_model.parameters(), lr=FINE_TUNE_LR) 

def fine_tune_model_loop(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        # Training Phase
        model.train()
        running_loss = 0.0
        train_iter = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [F-Tune Train]", leave=False)
        
        for inputs, labels in train_iter:
            inputs = inputs.to(DEVICE)
            labels = labels.float().unsqueeze(1).to(DEVICE) 
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        
        # Validation Phase
        model.eval()
        val_running_loss = 0.0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(DEVICE)
                labels = labels.float().unsqueeze(1).to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item() * inputs.size(0)

        val_epoch_loss = val_running_loss / len(val_loader.dataset)
        
        print(f"Epoch {epoch+1} Complete. Train Loss: {epoch_loss:.6f}, Val Loss: {val_epoch_loss:.6f}")
        
        # Save the best model based on validation loss
        if val_epoch_loss < best_val_loss:
            best_val_loss = val_epoch_loss
            torch.save(model.state_dict(), MODEL_SAVE_PATH)
            print(f"  -> Model improved and saved to {MODEL_SAVE_PATH}")
        
    print("Fine-Tuning Complete.")

# Start Fine-Tuning
# NOTE: Replace 'train_loader' and 'val_loader' with the actual objects from the first notebook
fine_tune_model_loop(fine_tune_ready_model, train_loader, val_loader, criterion, optimizer, FINE_TUNE_EPOCHS)


# --- 5. Final Evaluation and Metric Check (Goal 3.1 Prep) ---

# Load the best saved model for final testing
final_model = initialize_model(NUM_CLASSES)
final_model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))
final_model.to(DEVICE)
final_model.eval()

all_preds = []
all_labels = []

# NOTE: Replace 'test_loader' with the actual object from the first notebook
# with torch.no_grad():
#     for inputs, labels in test_loader:
#         inputs = inputs.to(DEVICE)
#         outputs = final_model(inputs).squeeze().cpu().numpy()
#         
#         # Convert probabilities to binary prediction (0 or 1)
#         preds = (outputs > 0.5).astype(int)
#         
#         all_preds.extend(preds)
#         all_labels.extend(labels.cpu().numpy())
# 
# accuracy = accuracy_score(all_labels, all_preds)
# precision = precision_score(all_labels, all_preds)
# recall = recall_score(all_labels, all_preds)
# f1 = f1_score(all_labels, all_preds)
# 
# print("\n--- Final Test Metrics ---")
# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall (Catching Defects): {recall:.4f}")
# print(f"F1-Score: {f1:.4f}")

Successfully loaded model weights from phase 1.
Total feature blocks: 19
Freezing blocks 0 to 15. Unfreezing blocks 16 to 18.
Trainable parameters: 1207361 / Total parameters: 2225153


Epoch 1 Complete. Train Loss: 0.155854, Val Loss: 0.082258
  -> Model improved and saved to final_best_model.pt


Epoch 2 Complete. Train Loss: 0.105622, Val Loss: 0.071266
  -> Model improved and saved to final_best_model.pt


Epoch 3 Complete. Train Loss: 0.084853, Val Loss: 0.054472
  -> Model improved and saved to final_best_model.pt


Epoch 4 Complete. Train Loss: 0.067746, Val Loss: 0.034669
  -> Model improved and saved to final_best_model.pt


Epoch 5 Complete. Train Loss: 0.057402, Val Loss: 0.030775
  -> Model improved and saved to final_best_model.pt
Fine-Tuning Complete.
